In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Dense
import numpy as np
import pandas as pd
import tensorflow as tf

# 실행 시마다 같은 결과값 도출을 위한 시드 설정
np.random.seed(0)
tf.random.set_seed(0)

# Tf

In [ ]:
# csv 파일
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)

# one hot encoding
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()

type_of_drink = encoder.fit_transform(train[['Type of Drink']]).toarray()
type_of_drink_col = encoder.get_feature_names_out(['Type of Drink'])

smoke = encoder.fit_transform(train[['Smoke']]).toarray()
smoke_col = encoder.get_feature_names_out(['Smoke'])

egfr_mutation = encoder.fit_transform(train[['EGFR mutation Detection']]).toarray()
egfr_mutation_col = encoder.get_feature_names_out(['EGFR mutation Detection'])

# stage 열은 엑셀에서 직접 암 병기를 계산
# 정확도에 큰 영향을 주지 못했다..
stage = encoder.fit_transform(train[['Stage']]).toarray()
stage_col = encoder.get_feature_names_out(['Stage'])

train.drop(['Type of Drink', 'Smoke', 'EGFR mutation Detection','Stage'], axis=1, inplace=True)

# 인코딩된 열 추가
train[type_of_drink_col] = type_of_drink
train[smoke_col] = smoke
train[egfr_mutation_col] = egfr_mutation
train[stage_col] = stage


type_of_drink = encoder.fit_transform(test[['Type of Drink']]).toarray()
type_of_drink_col = encoder.get_feature_names_out(['Type of Drink'])

smoke = encoder.fit_transform(test[['Smoke']]).toarray()
smoke_col = encoder.get_feature_names_out(['Smoke'])

egfr_mutation = encoder.fit_transform(test[['EGFR mutation Detection']]).toarray()
egfr_mutation_col = encoder.get_feature_names_out(['EGFR mutation Detection'])

# stage 열은 엑셀에서 직접 암 병기를 계산
# 정확도에 큰 영향을 주지 못했다..
stage = encoder.fit_transform(test[['Stage']]).toarray()
stage_col = encoder.get_feature_names_out(['Stage'])

test.drop(['Type of Drink', 'Smoke', 'EGFR mutation Detection','Stage'], axis=1, inplace=True)

# 인코딩된 열 추가
test[type_of_drink_col] = type_of_drink
test[smoke_col] = smoke
test[egfr_mutation_col] = egfr_mutation
test[stage_col] = stage

# sample
X_train = train.drop('Death', axis=1)
y_train = train['Death'] # Death label
X_test = test

from imblearn.over_sampling import BorderlineSMOTE

smote = BorderlineSMOTE()
X_train, y_train = smote.fit_resample(X_train, y_train)

from sklearn.preprocessing import PowerTransformer

scaler = PowerTransformer()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Nadam
import numpy as np

# 딥러닝 모델 구조 설정 (3개 층, 속성이 46개 input 값, swish와 sigmoid 활성화 함수 이용)
model = Sequential()
model.add(Dense(30, input_dim=46, activation='swish'))
model.add(Dense(15, activation='swish'))
model.add(Dense(1, activation='sigmoid'))

# 학습률 조절
learning_rate = 0.0001
optimizer = Nadam(learning_rate=learning_rate)

# 딥러닝 실행 (오차함수는 binary_crossentropy, 최적화함수는 Nadam 이용)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# 배치 크기와 에포크 수 조절
batch_size = 10
epochs = 50

# 모델 훈련
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

# 만들어진 모델 저장
model.save('Predict_Model.h5')

# 모델 불러오기
model = load_model('Predict_Model.h5')

# X 값에 대한 predict_classes 함수 실행결과 Y에 저장
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype("int32")

print('Predict : ', y_pred)

# 예측 값의 분포 확인
print(np.unique(y_pred, return_counts=True))


Epoch 1/50
1580/1580 [==============================] - 7s 3ms/step - loss: 0.6318 - accuracy: 0.6677
Epoch 2/50
1580/1580 [==============================] - 4s 3ms/step - loss: 0.4793 - accuracy: 0.7949
Epoch 3/50
1580/1580 [==============================] - 5s 3ms/step - loss: 0.4141 - accuracy: 0.8270
Epoch 4/50
1580/1580 [==============================] - 5s 3ms/step - loss: 0.3872 - accuracy: 0.8395
Epoch 5/50
1580/1580 [==============================] - 4s 3ms/step - loss: 0.3728 - accuracy: 0.8456
Epoch 6/50
1580/1580 [==============================] - 5s 3ms/step - loss: 0.3640 - accuracy: 0.8499
Epoch 7/50
1580/1580 [==============================] - 5s 3ms/step - loss: 0.3581 - accuracy: 0.8506
Epoch 8/50
1580/1580 [==============================] - 4s 2ms/step - loss: 0.3538 - accuracy: 0.8523
Epoch 9/50
1580/1580 [==============================] - 4s 3ms/step - loss: 0.3505 - accuracy: 0.8530
Epoch 10/50
1580/1580 [==============================] - 5s 3ms/step - loss: 0.347

In [ ]:
result = pd.read_csv('/content/submission.csv', encoding = 'cp949')# 파일 넣기
submission_id = result['ID']
submission_label = pd.DataFrame(y_pred,columns = ['Death'])
submission = pd.concat([submission_id,submission_label],axis=1)
submission.to_csv('result.csv', index = False, encoding = 'utf-8-sig')

# 1 try

In [ ]:
# csv 파일
train = pd.read_csv('strain.csv', index_col=0)
test = pd.read_csv('stest.csv', index_col=0)

In [ ]:
# one hot encoding
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()

In [ ]:
type_of_drink = encoder.fit_transform(train[['Type of Drink']]).toarray()
type_of_drink_col = encoder.get_feature_names_out(['Type of Drink'])

smoke = encoder.fit_transform(train[['Smoke']]).toarray()
smoke_col = encoder.get_feature_names_out(['Smoke'])

egfr_mutation = encoder.fit_transform(train[['EGFR mutation Detection']]).toarray()
egfr_mutation_col = encoder.get_feature_names_out(['EGFR mutation Detection'])

stage = encoder.fit_transform(train[['Stage']]).toarray()
stage_col = encoder.get_feature_names_out(['Stage'])

train.drop(['Type of Drink', 'Smoke', 'EGFR mutation Detection','Stage'], axis=1, inplace=True)

# 인코딩된 열 추가
train[type_of_drink_col] = type_of_drink
train[smoke_col] = smoke
train[egfr_mutation_col] = egfr_mutation
train[stage_col] = stage


type_of_drink = encoder.fit_transform(test[['Type of Drink']]).toarray()
type_of_drink_col = encoder.get_feature_names_out(['Type of Drink'])

smoke = encoder.fit_transform(test[['Smoke']]).toarray()
smoke_col = encoder.get_feature_names_out(['Smoke'])

egfr_mutation = encoder.fit_transform(test[['EGFR mutation Detection']]).toarray()
egfr_mutation_col = encoder.get_feature_names_out(['EGFR mutation Detection'])

stage = encoder.fit_transform(test[['Stage']]).toarray()
stage_col = encoder.get_feature_names_out(['Stage'])

test.drop(['Type of Drink', 'Smoke', 'EGFR mutation Detection','Stage'], axis=1, inplace=True)

# 인코딩된 열 추가
test[type_of_drink_col] = type_of_drink
test[smoke_col] = smoke
test[egfr_mutation_col] = egfr_mutation
test[stage_col] = stage


In [ ]:
# sample
X_train = train.drop('Death', axis=1)
y_train = train['Death'] # Death label
X_test = test

In [ ]:
from imblearn.over_sampling import BorderlineSMOTE

smote = BorderlineSMOTE()
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
from sklearn.preprocessing import PowerTransformer

scaler = PowerTransformer()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# 딥러닝 모델 구조 설정(3개층, 속성이 17개 input 값, relu와 sigmoid 활성화 함수 이용)
model = Sequential()
model.add(Dense(30, input_dim=46, activation='swish'))
model.add(Dense(15, activation='swish'))
model.add(Dense(1, activation='sigmoid'))

# 딥러닝 실행(오차함수는 평균제곱오차, 최적화함수는 adam 이용)
model.compile(loss='binary_crossentropy', optimizer='Nadam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=5)

# 만들어진 모델 저장
model.save('Predict_Model.h5')

Epoch 1/100
3159/3159 [==============================] - 14s 3ms/step - loss: 0.3932 - accuracy: 0.8277
Epoch 2/100
3159/3159 [==============================] - 7s 2ms/step - loss: 0.3424 - accuracy: 0.8555
Epoch 3/100
3159/3159 [==============================] - 6s 2ms/step - loss: 0.3367 - accuracy: 0.8571
Epoch 4/100
3159/3159 [==============================] - 7s 2ms/step - loss: 0.3330 - accuracy: 0.8602
Epoch 5/100
3159/3159 [==============================] - 6s 2ms/step - loss: 0.3285 - accuracy: 0.8615
Epoch 6/100
3159/3159 [==============================] - 7s 2ms/step - loss: 0.3255 - accuracy: 0.8612
Epoch 7/100
3159/3159 [==============================] - 6s 2ms/step - loss: 0.3209 - accuracy: 0.8639
Epoch 8/100
3159/3159 [==============================] - 7s 2ms/step - loss: 0.3167 - accuracy: 0.8635
Epoch 9/100
3159/3159 [==============================] - 6s 2ms/step - loss: 0.3129 - accuracy: 0.8655
Epoch 10/100
3159/3159 [==============================] - 7s 2ms/step - 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Nadam

# 모델 구조 설정 (4개 층, Dropout 및 BatchNormalization 추가)
model = Sequential()
model.add(Dense(64, input_dim=39, activation='swish'))
model.add(BatchNormalization())
model.add(Dropout(0.2))  # Dropout 비율 조정
model.add(Dense(32, activation='swish'))
model.add(BatchNormalization())
model.add(Dropout(0.2))  # Dropout 비율 조정
model.add(Dense(16, activation='swish'))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일 (오차함수는 binary_crossentropy, 최적화함수는 adam 이용)
optimizer = Nadam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)  # 모니터링 대상 변경

# 딥러닝 실행 (훈련 및 검증 데이터로 나눠서 사용)
history = model.fit(X_train, y_train, epochs=100, batch_size=5, validation_split=0.2, callbacks=[early_stopping])  # 검증 데이터 비율 조정

# 만들어진 모델 저장
model.save('Predict_Model.h5')


Epoch 1/100
2527/2527 [==============================] - 18s 6ms/step - loss: 0.5603 - accuracy: 0.7336 - val_loss: 0.1874 - val_accuracy: 0.8797
Epoch 2/100
2527/2527 [==============================] - 14s 6ms/step - loss: 0.4849 - accuracy: 0.7883 - val_loss: 0.1670 - val_accuracy: 0.8797
Epoch 3/100
2527/2527 [==============================] - 14s 6ms/step - loss: 0.4679 - accuracy: 0.7959 - val_loss: 0.1500 - val_accuracy: 0.8883
Epoch 4/100
2527/2527 [==============================] - 15s 6ms/step - loss: 0.4631 - accuracy: 0.8006 - val_loss: 0.1480 - val_accuracy: 0.8848
Epoch 5/100
2527/2527 [==============================] - 16s 6ms/step - loss: 0.4531 - accuracy: 0.8047 - val_loss: 0.1341 - val_accuracy: 0.9006
Epoch 6/100
2527/2527 [==============================] - 16s 6ms/step - loss: 0.4533 - accuracy: 0.8033 - val_loss: 0.1409 - val_accuracy: 0.9006
Epoch 7/100
2527/2527 [==============================] - 14s 6ms/step - loss: 0.4495 - accuracy: 0.8070 - val_loss: 0.1331 -

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# 딥러닝에 필요한 케라스 함수 호출
from keras.models import load_model

# 모델 불러오기
model = load_model('Predict_Model.h5')

# X 값에 대한 predict_classes 함수 실행결과 Y에 저장
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype("int32")

print('Predict : ', y_pred)

157/157 [==============================] - 0s 1ms/step
Predict :  [[0]
 [1]
 [0]
 ...
 [0]
 [0]
 [1]]


In [ ]:
np.unique(y_pred, return_counts=True)

(array([0, 1], dtype=int32), array([4256,  744]))

In [ ]:
result = pd.read_csv('/content/submission.csv', encoding = 'cp949')# 파일 넣기
submission_id = result['ID']
submission_label = pd.DataFrame(y_pred,columns = ['Death'])
submission = pd.concat([submission_id,submission_label],axis=1)
submission.to_csv('result.csv', index = False, encoding = 'utf-8-sig')

In [ ]:
# 딥러닝 모델 구조 설정(3개층, 속성이 17개 input 값, relu와 sigmoid 활성화 함수 이용)
model = Sequential()
model.add(Dense(30, input_dim=39, activation='swish'))
model.add(Dense(15, activation='swish'))
model.add(Dense(1, activation='sigmoid'))

# 딥러닝 실행(오차함수는 평균제곱오차, 최적화함수는 adam 이용)
model.compile(loss='binary_crossentropy', optimizer='Nadam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=5)

# 만들어진 모델 저장
model.save('Predict_Model.h5')

Epoch 1/100
3159/3159 [==============================] - 14s 4ms/step - loss: 0.4045 - accuracy: 0.8189
Epoch 2/100
3159/3159 [==============================] - 13s 4ms/step - loss: 0.3570 - accuracy: 0.8456
Epoch 3/100
3159/3159 [==============================] - 13s 4ms/step - loss: 0.3491 - accuracy: 0.8482
Epoch 4/100
3159/3159 [==============================] - 12s 4ms/step - loss: 0.3424 - accuracy: 0.8526
Epoch 5/100
3159/3159 [==============================] - 12s 4ms/step - loss: 0.3378 - accuracy: 0.8542
Epoch 6/100
3159/3159 [==============================] - 13s 4ms/step - loss: 0.3339 - accuracy: 0.8567
Epoch 7/100
3159/3159 [==============================] - 12s 4ms/step - loss: 0.3297 - accuracy: 0.8568
Epoch 8/100
3159/3159 [==============================] - 12s 4ms/step - loss: 0.3254 - accuracy: 0.8574
Epoch 9/100
3159/3159 [==============================] - 14s 4ms/step - loss: 0.3224 - accuracy: 0.8603
Epoch 10/100
3159/3159 [==============================] - 15s 5m

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
